In [54]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from mistralai.models.jobs import TrainingParameters
import json
import pandas as pd

api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)
model= 'mistral-small-2402'

In [36]:
messages = [
    ChatMessage(role="user", 
                content="Qu'est-ce que le machine learning ?")
]

response = client.chat(
    model=model,
    messages=messages
)

chat_response = response.dict()['choices'][0]['message']['content']

In [37]:
chat_response

"Le machine learning, ou apprentissage automatique en français, est une branche de l'intelligence artificielle qui permet aux ordinateurs et aux machines d'apprendre à partir de données et d'améliorer leurs performances sur une tâche spécifique sans être explicitement programmés.\n\nIl existe plusieurs types de machine learning, notamment l'apprentissage supervisé, l'apprentissage non supervisé et l'apprentissage par renforcement. L'apprentissage supervisé consiste à entraîner un modèle à partir de données étiquetées, c'est-à-dire de données pour lesquelles la réponse correcte est déjà connue. L'apprentissage non supervisé, quant à lui, consiste à entraîner un modèle à partir de données non étiquetées, c'est-à-dire de données pour lesquelles la réponse correcte n'est pas connue. L'apprentissage par renforcement consiste à entraîner un modèle à partir d'une récompense ou d'une pénalité reçue en fonction de ses actions.\n\nLe machine learning a de nombreuses applications, notamment dans 

In [90]:
df = pd.DataFrame()

In [91]:
df1 = pd.read_csv('stackoverflow1.csv').dropna()
df2 = pd.read_csv('stackoverflow2.csv').dropna()
df3 = pd.read_csv('stackoverflow3.csv').dropna()

In [92]:
df = pd.concat([df1, df2, df3])

In [93]:
def replace_str(_df: pd.DataFrame, _column: str):
    _df[_column] = _df[_column].str.replace('\n', ' ')
    _df[_column] = _df[_column].str.replace('\r', ' ')
    _df[_column] = _df[_column].str.replace('\t', ' ')
    _df[_column] = _df[_column].str.lower()\
        .str.replace(r'[^\w\s/<>=]', '')\
        .str.replace(r'\s+', ' ')
    return _df

In [94]:
def pre_process_df(_df: pd.DataFrame):
    _df = _df.dropna()
    _df.drop_duplicates(inplace=True)
    _df.reset_index(inplace=True, drop=True)
    _df['answer'] = _df['answer'].str.split('\nShare').str[0]
    _df = replace_str(_df, 'answer')
    _df = replace_str(_df, 'question')
    return _df

In [95]:
df = pre_process_df(df)

In [96]:
def count_tokens(text):
    if pd.isna(text):
        return 0
    return len(text.split())

token_counts = df.applymap(count_tokens)

total_token_count = token_counts.sum().sum()

print(f'Total number of tokens in the DataFrame: {total_token_count}')

Total number of tokens in the DataFrame: 3706313


/var/folders/vf/qcsfk0590zj_zrqnrs3xwnxr0000gn/T/ipykernel_5035/2006679773.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  token_counts = df.applymap(count_tokens)


In [97]:
data_list = []
for index, row in df.iterrows():
    data_list.append({
        "rôle": 'user',
        "output": row['question']
    })
    data_list.append({
        "rôle": 'assistant',
        "output": row['answer']
    })

In [98]:
df.to_parquet('stackoverflow.parquet')

In [99]:
df = pd.read_parquet('stackoverflow.parquet')

df_train = df.sample(frac=0.9)
df_eval = df.drop(df_train.index)

In [108]:
jsonl_path = 'training_file.jsonl'

In [109]:
records = df_train.to_dict(orient='records')

with open(jsonl_path, 'w') as file:
    for index, row in enumerate(records):
        user_message = {
            "role": "user",
            "content": row['question']
        }
        assistant_message = {
            "role": "assistant",
            "content": row['answer']
        }
        jsonl_format = {"messages": [user_message, assistant_message]}
        file.write(json.dumps(jsonl_format) + '\n')

print(f'JSONL file saved at {jsonl_path}')

JSONL file saved at training_file.jsonl


In [110]:
jsonl_path = 'evaluation_file.jsonl'

In [111]:
records = df_eval.to_dict(orient='records')

with open(jsonl_path, 'w') as file:
    for index, row in enumerate(records):
        user_message = {
            "role": "user",
            "content": row['question']
        }
        assistant_message = {
            "role": "assistant",
            "content": row['answer']
        }
        jsonl_format = {"messages": [user_message, assistant_message]}
        file.write(json.dumps(jsonl_format) + '\n')

print(f'JSONL file saved at {jsonl_path}')

JSONL file saved at evaluation_file.jsonl


In [112]:
with open("training_file.jsonl", "rb") as f:
    training_data = client.files.create(file=("training_file.jsonl", f)) 

print("Training file uploaded")

with open("evaluation_file.jsonl", "rb") as f:
    evaluation_data = client.files.create(file=("evaluation_file.jsonl", f)) 

print("Evaluation file uploaded")

Training file uploaded
Evaluation file uploaded


In [114]:
created_jobs = client.jobs.create(
        model="open-mistral-7b",
        training_files=[training_data.id],
        validation_files=[evaluation_data.id],
        hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)
created_jobs

MistralAPIException: Status: 422. Message: {"detail": "The given validation set is too big. It should not exceed more than 1000000 bytes, but it is 3384199."}

In [ ]:
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
print(retrieved_jobs)

In [ ]:
chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[ChatMessage(role='user', content="Qu'est-ce que le scrapping ?")]
)